# Periodic Table of Elements

Source:

- [original notebook](https://plotnine.readthedocs.io/en/v0.12.4/generated/plotnine.geoms.geom_tile.html#periodic-table-of-elements),

- [dataset](https://gist.github.com/GoodmanSciences/c2dd862cd38f21b0ad36b8f96b4bf1ee#file-periodic-table-of-elements-csv).

In [1]:
from lets_plot import *

In [2]:
LetsPlot.setup_html()

In [3]:
def get_elements_df():
    import pandas as pd
    
    return pd.read_csv("https://gist.githubusercontent.com/GoodmanSciences/c2dd862cd38f21b0ad36b8f96b4bf1ee/raw/1d92663004489a5b6926e944c1b3d9ec5c40900e/Periodic%2520Table%2520of%2520Elements.csv")

elements_df = get_elements_df()
print(elements_df.shape)
elements_df.head()

(118, 28)


,AtomicNumber,Element,Symbol,AtomicMass,NumberofNeutrons,NumberofProtons,NumberofElectrons,Period,Group,Phase,...,FirstIonization,Density,MeltingPoint,BoilingPoint,NumberOfIsotopes,Discoverer,Year,SpecificHeat,NumberofShells,NumberofValence
0,1,Hydrogen,H,1.007,0,1,1,1,1.0,gas,...,13.5984,0.000090,14.175,20.28,3.0,Cavendish,1766.0,14.304,1,1.0
1,2,Helium,He,4.002,2,2,2,1,18.0,gas,...,24.5874,0.000179,NaN,4.22,5.0,Janssen,1868.0,5.193,1,NaN
2,3,Lithium,Li,6.941,4,3,3,2,1.0,solid,...,5.3917,0.534000,453.850,1615.00,5.0,Arfvedson,1817.0,3.582,2,1.0
3,4,Beryllium,Be,9.012,5,4,4,2,2.0,solid,...,9.3227,1.850000,1560.150,2742.00,6.0,Vaulquelin,1798.0,1.825,2,2.0
4,5,Boron,B,10.811,6,5,5,2,13.0,solid,...,8.2980,2.340000,2573.150,4200.00,6.0,Gay-Lussac,1808.0,1.026,2,3.0


In [4]:
bottom_filter = lambda df: ((df["Type"] == "Actinide")&(df["Element"] != "Radium"))|(df["Type"] == "Lanthanide")

In [5]:
def prepare_top_df(df):
    return df[~bottom_filter(df)].assign(
        X=lambda df: df["Group"],
        Y=lambda df: df["Period"],
    )

top_df = prepare_top_df(elements_df)
top_df.shape

(88, 30)

In [6]:
def prepare_bottom_df(df):
    import numpy as np

    result = df[bottom_filter(df)]
    nrows = 2
    hshift = 2.5
    vshift = 3
    return result.assign(
        X=np.tile(np.arange(len(result) // nrows), nrows) + hshift,
        Y=result["Period"] + vshift
    )

bottom_df = prepare_bottom_df(elements_df)
bottom_df.shape

(30, 30)

In [7]:
tile_side = .95
tile_ratio = 1.2

First peak:

In [8]:
ggplot(mapping=aes("X", "Y")) + \
    geom_tile(data=top_df, width=tile_side, height=tile_side) + \
    geom_tile(data=bottom_df, width=tile_side, height=tile_side) + \
    coord_fixed(ratio=tile_ratio)

The table upside down. Fix that with a scale:

In [9]:
ggplot(mapping=aes("X", "Y")) + \
    geom_tile(data=top_df, width=tile_side, height=tile_side) + \
    geom_tile(data=bottom_df, width=tile_side, height=tile_side) + \
    scale_y_reverse() + \
    coord_fixed(ratio=tile_ratio)

Coloring:

In [10]:
ggplot(mapping=aes("X", "Y", fill="Type")) + \
    geom_tile(data=top_df, width=tile_side, height=tile_side) + \
    geom_tile(data=bottom_df, width=tile_side, height=tile_side) + \
    scale_y_reverse() + \
    coord_fixed(ratio=tile_ratio)

Add text to the tiles:

In [11]:
def inner_text(df):
    return geom_text(aes(label="AtomicNumber"), data=df, nudge_x=-0.4, nudge_y=0.4, hjust='left', vjust='top', size=4) + \
        geom_text(aes(label="Symbol"), data=df, nudge_y=.1, size=6, fontface='bold') + \
        geom_text(aes(label="Element"), data=df, nudge_y=-0.125, size=3) + \
        geom_text(aes(label="AtomicMass"), data=df, nudge_y=-.3, size=3)

In [12]:
ggplot(mapping=aes("X", "Y", fill="Type")) + \
    geom_tile(data=top_df, width=tile_side, height=tile_side) + \
    geom_tile(data=bottom_df, width=tile_side, height=tile_side) + \
    inner_text(top_df) + \
    inner_text(bottom_df) + \
    scale_y_reverse() + \
    coord_fixed(ratio=tile_ratio)

Resize:

In [13]:
ggplot(mapping=aes("X", "Y", fill="Type")) + \
    geom_tile(data=top_df, width=tile_side, height=tile_side) + \
    geom_tile(data=bottom_df, width=tile_side, height=tile_side) + \
    inner_text(top_df) + \
    inner_text(bottom_df) + \
    scale_y_reverse() + \
    coord_fixed(ratio=tile_ratio) + \
    ggsize(1000, 700)

Change fill palette and theme:

In [14]:
ggplot(mapping=aes("X", "Y", fill="Type")) + \
    geom_tile(data=top_df, width=tile_side, height=tile_side) + \
    geom_tile(data=bottom_df, width=tile_side, height=tile_side) + \
    inner_text(top_df) + \
    inner_text(bottom_df) + \
    scale_y_reverse() + \
    scale_fill_brewer(type='qual', palette='Set2') + \
    coord_fixed(ratio=tile_ratio) + \
    ggsize(1000, 700) + \
    theme_void()

Add the group number along the top most row of each column, and period number along the left side of the top table:

In [15]:
def get_group_df(df):
    result = df.groupby(
        "Group"
    ).agg(
        Y=("Period", 'min')
    ).reset_index()
    result["Group"] = result["Group"].astype(int)
    return result

group_df = get_group_df(top_df)

In [16]:
def get_period_df(min_value, max_value):
    return {
        "X": [0] * (max_value - min_value + 1),
        "Period": list(range(min_value, max_value + 1)),
    }

period_df = get_period_df(1, 7)

In [17]:
ggplot(mapping=aes("X", "Y", fill="Type")) + \
    geom_tile(data=top_df, width=tile_side, height=tile_side) + \
    geom_tile(data=bottom_df, width=tile_side, height=tile_side) + \
    inner_text(top_df) + \
    inner_text(bottom_df) + \
    geom_text(aes("Group", "Y", label="Group"), data=group_df, \
              color='gray', nudge_y=.525, vjust='bottom', size=6) + \
    geom_text(aes("X", "Period", label="Period"), data=period_df, \
              color='gray', nudge_x=.375, vjust='right', size=6) + \
    scale_y_reverse() + \
    scale_fill_brewer(type='qual', palette='Set2') + \
    coord_fixed(ratio=tile_ratio) + \
    ggsize(1000, 700) + \
    theme_void()